### transfermarkt 선수가치 크롤러

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
from concurrent.futures import ThreadPoolExecutor # 병렬로 수집

1. 포지션별 선수가치

In [6]:
headers = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
}

position_urls = {
    'forwards': 'Sturm',
    'midfielders': 'Mittelfeld',
    'defenders': 'Abwehr',
    'goalkeepers': 'Torwart'
}

all_positions = []

for position_key, position_value in position_urls.items():
    for page in range(1, 21): 
        url = f'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop/plus/ausrichtung/{position_value}/spielerposition_id/alle/altersklasse/alle/jahrgang/0/land_id/0/kontinent_id/0/yt0/Anzeigen/0//page/{page}?ajax=yw1'
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.content, 'html.parser')
        player_info = soup.find_all('tr', {'class': ['odd', 'even']})

        for info in player_info:
            player = info.find_all('td')
            all_positions.append({
                'name': player[3].text.strip(),
                'position': player[4].text.strip(),
                'age': player[5].text.strip(),
                'nation': player[6].img['alt'] if player[6].find('img') else '',
                'club': player[7].img['alt'] if player[7].find('img') else '',
                'value': player[8].text.strip()
                })


all_positions_df = pd.DataFrame(all_positions)
all_positions_df = all_positions_df.drop_duplicates()

all_positions_df.to_csv('../data/transfermarkt_all_positions.csv', index=False, encoding='utf-8-sig')
all_positions_df

,name,position,age,nation,club,value
0,Lamine Yamal,Right Winger,18,Spain,FC Barcelona,€200.00m
1,Erling Haaland,Centre-Forward,25,Norway,Manchester City,€180.00m
2,Kylian Mbappé,Centre-Forward,26,France,Real Madrid,€180.00m
3,Vinicius Junior,Left Winger,25,Brazil,Real Madrid,€170.00m
4,Bukayo Saka,Right Winger,23,England,Arsenal FC,€150.00m
...,...,...,...,...,...,...
1995,Brad Collins,Goalkeeper,28,England,Coventry City,€1.00m
1996,Juan Soriano,Goalkeeper,27,Spain,CD Leganés,€1.00m
1997,Alex Palmer,Goalkeeper,28,England,Ipswich Town,€1.00m
1998,Pierluigi Gollini,Goalkeeper,30,Italy,AS Roma,€1.00m


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
}

position_urls = {
    'forwards': 'Sturm',
    'midfielders': 'Mittelfeld',
    'defenders': 'Abwehr',
    'goalkeepers': 'Torwart'
}

all_players = []

for position_key, position_value in position_urls.items():
    for page in range(1, 21):
        url = f'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop/plus/ausrichtung/{position_value}/spielerposition_id/alle/altersklasse/alle/jahrgang/0/land_id/0/kontinent_id/0/yt0/Anzeigen/0//page/{page}?ajax=yw1'
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.content, 'html.parser')
        player_rows = soup.find_all('tr', {'class': ['odd', 'even']})

        for row in player_rows:
            player = row.find_all('td')
            if len(player) < 9:
                continue

            link_tag = player[3].find('a')
            profile_href = link_tag['href'] if link_tag else ''
            profile_url = 'https://www.transfermarkt.com' + profile_href

            # 상세페이지 요청
            profile_res = requests.get(profile_url, headers=headers)
            profile_soup = BeautifulSoup(profile_res.content, 'html.parser')

            # height
            try:
                height_tag = profile_soup.find('span', text='Height:').find_next('span')
                height = height_tag.text.strip()
            except:
                height = ''

            # foot
            try:
                foot_tag = profile_soup.find('span', text='Foot:').find_next('span')
                foot = foot_tag.text.strip()
            except:
                foot = ''

            # player agent
            try:
                agent_tag = profile_soup.find('span', text='Player agent:').find_next('a')
                agent = agent_tag.text.strip()
            except:
                agent = ''

            # highest market value
            try:
                max_val_div = profile_soup.find('div', class_='max-value svelte-gfmgwx')
                highest_value = max_val_div.text.strip()
            except:
                highest_value = ''

            all_players.append({
                'name': player[3].text.strip(),
                'position': player[4].text.strip(),
                'age': player[5].text.strip(),
                'nation': player[6].img['alt'] if player[6].find('img') else '',
                'club': player[7].img['alt'] if player[7].find('img') else '',
                'value': player[8].text.strip(),
                'height': height,
                'foot': foot,
                'player_agent': agent,
                'highest_value': highest_value
            })

            time.sleep(0.5)  # 차단 방지용 지연

# 데이터프레임으로 저장
df = pd.DataFrame(all_players)
df = df.drop_duplicates()
df.to_csv('../data/transfermarkt_all_positions_extended.csv', index=False, encoding='utf-8-sig')


2. 유망주 선수가치

In [ ]:
# 원본 코드 -> 건들지 말것
risingstars_data = []

for i in range(1,21):
    url = f'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop/plus/ausrichtung/alle/spielerposition_id/alle/altersklasse/u21/jahrgang/0/land_id/0/kontinent_id/0/yt0/Anzeigen/0//page/{i}?ajax=yw1'
    res = requests.get(url, headers={
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
    })
    soup = BeautifulSoup(res.content, 'html.parser')
    player_info = soup.find_all('tr', {'class': ['odd', 'even']})

    for info in player_info:
        player = info.find_all('td')
        risingstars_data.append({
            'name': player[3].text.strip(),
            'position': player[4].text.strip(),
            'age': player[5].text.strip(),
            'nation': player[6].img['alt'] if player[6].find('img') else '',
            'club': player[7].img['alt'] if player[7].find('img') else '',
            'value': player[8].text.strip()
            })

risingstars_df = pd.DataFrame(risingstars_data)
risingstars_df.to_csv('../data/transfermarkt_risingstars.csv', index=False, encoding='utf-8-sig')

risingstars_df

,name,position,age,nation,club,value
0,Lamine Yamal,Right Winger,18,Spain,FC Barcelona,€200.00m
1,Désiré Doué,Right Winger,20,France,Paris Saint-Germain,€90.00m
2,Pau Cubarsí,Centre-Back,18,Spain,FC Barcelona,€80.00m
3,João Neves,Defensive Midfield,20,Portugal,Paris Saint-Germain,€80.00m
4,Estêvão,Right Winger,18,Brazil,Chelsea FC,€60.00m
...,...,...,...,...,...,...
495,Maurice Krattenmacher,Attacking Midfield,19,Germany,Hertha BSC,€3.50m
496,Max Dean,Centre-Forward,21,England,KAA Gent,€3.50m
497,Charlie Hughes,Centre-Back,21,England,Hull City,€3.50m
498,Cihan Çanak,Right Winger,20,Türkiye,Trabzonspor,€3.50m


In [6]:
def parse_height(text):     # 예: "1,73 m" → "173"
    match = re.search(r'(\d+),(\d+)', text)
    if match:
        meters = int(match.group(1))
        centimeters = int(match.group(2))
        return str(meters * 100 + centimeters)
    return ''

session = requests.Session()
session.headers.update({
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
})

risingstars_data = []

for i in range(1, 21):
    url = f'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop/plus/ausrichtung/alle/spielerposition_id/alle/altersklasse/u21/jahrgang/0/land_id/0/kontinent_id/0/yt0/Anzeigen/0//page/{i}?ajax=yw1'
    res = session.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    player_info = soup.find_all('tr', {'class': ['odd', 'even']})

    for info in player_info:
        player = info.find_all('td')
        if len(player) < 9:
            continue

        name = player[3].text.strip()
        position = player[4].text.strip()
        age = player[5].text.strip()
        nation = player[6].img['alt'] if player[6].find('img') else ''
        club = player[7].img['alt'] if player[7].find('img') else ''
        value = player[8].text.strip()

        height = foot = agent = ''
        player_url_tag = player[3].find('a')
        player_url = 'https://www.transfermarkt.com' + player_url_tag['href'] if player_url_tag else ''

        if player_url:
            try:
                detail_res = session.get(player_url)
                detail_soup = BeautifulSoup(detail_res.content, 'html.parser')

                height_tag = detail_soup.find('span', string='Height:')
                foot_tag = detail_soup.find('span', string='Foot:')
                agent_tag = detail_soup.find('span', string='Player agent:')

                height_raw = height_tag.find_next('span').text.strip() if height_tag else ''
                height = parse_height(height_raw)

                foot = foot_tag.find_next('span').text.strip() if foot_tag else ''
                agent = agent_tag.find_next('a').text.strip() if agent_tag else ''

                time.sleep(0.7)  # 상세 페이지 접근 시 약간의 딜레이
            except:
                pass

        risingstars_data.append({
            'name': name,
            'position': position,
            'age': age,
            'height': height,
            'foot': foot,
            'nation': nation,
            'club': club,
            'value': value,
            'player_agent': agent
        })

    time.sleep(0.7)  # 리스트 페이지 접근 시 딜레이

# 저장
risingstars_df = pd.DataFrame(risingstars_data)
risingstars_df.to_csv('../data/transfermarkt_risingstars_extended.csv', index=False, encoding='utf-8-sig')
risingstars_df

,name,position,age,height,foot,nation,club,value,player_agent
0,Lamine Yamal,Right Winger,18,180,left,Spain,FC Barcelona,€200.00m,Gestifute
1,Désiré Doué,Right Winger,20,181,right,France,Paris Saint-Germain,€90.00m,
2,Pau Cubarsí,Centre-Back,18,184,right,Spain,FC Barcelona,€80.00m,Bahía Internacional
3,João Neves,Defensive Midfield,20,174,right,Portugal,Paris Saint-Germain,€80.00m,Gestifute
4,Estêvão,Right Winger,18,176,left,Brazil,Chelsea FC,€60.00m,LINK SPORTS
...,...,...,...,...,...,...,...,...,...
495,Cihan Çanak,Right Winger,20,,,Türkiye,Trabzonspor,€3.50m,
496,Alfie Devine,Attacking Midfield,20,,,England,Tottenham Hotspur,€3.50m,
497,Brooke Norton-Cuffy,Right-Back,21,181,right,England,Genoa CFC,€3.50m,Roc Nation Sports
498,Andrew Moran,Attacking Midfield,21,,,Ireland,Brighton & Hove Albion,€3.50m,


In [2]:
# 헤더 설정
headers = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
}

# 높이 정규화 함수
def parse_height(text):
    match = re.search(r"(\d+),(\d+)", text)
    if match:
        return str(int(match.group(1)) * 100 + int(match.group(2)))
    return ""

# 1단계: 기본 정보 수집
basic_info_list = []

for i in range(1, 11):  # 50명 x 10페이지 = 500명
    url = f"https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop/plus/ausrichtung/alle/spielerposition_id/alle/altersklasse/alle/jahrgang/0/land_id/0/kontinent_id/0/yt0/Anzeigen/0//page/{i}?ajax=yw1"
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.content, "html.parser")
    rows = soup.find_all("tr", class_=["odd", "even"])

    for row in rows:
        tds = row.find_all("td")
        if len(tds) < 9:
            continue
        link_tag = tds[3].find("a")
        player_url = "https://www.transfermarkt.com" + link_tag["href"] if link_tag else ""
        basic_info_list.append({
            "name": tds[3].text.strip(),
            "position": tds[4].text.strip(),
            "age": tds[5].text.strip(),
            "nation": tds[6].img["alt"] if tds[6].find("img") else "",
            "club": tds[7].img["alt"] if tds[7].find("img") else "",
            "value": tds[8].text.strip(),
            "url": player_url
        })

# 2단계: 병렬로 상세 정보 수집
def fetch_detail(player):
    try:
        res = requests.get(player["url"], headers=headers)
        soup = BeautifulSoup(res.content, "html.parser")
        height_tag = soup.find("span", string="Height:")
        foot_tag = soup.find("span", string="Foot:")
        agent_tag = soup.find("span", string="Player agent:")

        player["height"] = parse_height(height_tag.find_next("span").text.strip()) if height_tag else ""
        player["foot"] = foot_tag.find_next("span").text.strip() if foot_tag else ""
        player["player_agent"] = agent_tag.find_next("a").text.strip() if agent_tag else ""
    except:
        player["height"] = player["foot"] = player["player_agent"] = ""
    return player

with ThreadPoolExecutor(max_workers=10) as executor:
    detailed_players = list(executor.map(fetch_detail, basic_info_list))

# 3단계: 저장
risingstars_df = pd.DataFrame(detailed_players)

risingstars_df.to_csv("../data/transfermarkt_optimized_500players.csv", index=False, encoding="utf-8-sig")
risingstars_df

,name,position,age,nation,club,value,url,height,foot,player_agent
0,Lamine Yamal,Right Winger,18,Spain,FC Barcelona,€200.00m,https://www.transfermarkt.com/lamine-yamal/pro...,180,left,Gestifute
1,Jude Bellingham,Attacking Midfield,22,England,Real Madrid,€180.00m,https://www.transfermarkt.com/jude-bellingham/...,186,right,
2,Erling Haaland,Centre-Forward,25,Norway,Manchester City,€180.00m,https://www.transfermarkt.com/erling-haaland/p...,195,left,Rafaela Pimenta
3,Kylian Mbappé,Centre-Forward,26,France,Real Madrid,€180.00m,https://www.transfermarkt.com/kylian-mbappe/pr...,178,right,
4,Vinicius Junior,Left Winger,25,Brazil,Real Madrid,€170.00m,https://www.transfermarkt.com/vinicius-junior/...,176,right,Roc Nation Sports
...,...,...,...,...,...,...,...,...,...,...
245,Tomás Araújo,Centre-Back,23,Portugal,SL Benfica,€32.00m,https://www.transfermarkt.com/tomas-araujo/pro...,,,
246,Jacob Ramsey,Left Midfield,24,England,Aston Villa,€32.00m,https://www.transfermarkt.com/jacob-ramsey/pro...,,,
247,Josip Stanisic,Right-Back,25,Croatia,Bayern Munich,€32.00m,https://www.transfermarkt.com/josip-stanisic/p...,,,
248,Malik Tillman,Attacking Midfield,23,United States,Bayer 04 Leverkusen,€32.00m,https://www.transfermarkt.com/malik-tillman/pr...,,,


3. 대륙 별 선수가치

In [9]:
headers = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
}

confederation_ids = {
    'UEFA': 1, # 유럽 축구 연맹
    'AFC': 2, # 아시아 축구 연맹
    'CAF': 3, # 아프리카 축구 연맹
    'CONCACAF': 4, # 북중미 축구 연맹
    'CONMEBOL': 5, # 남미 축구 연맹
    'OFC': 6 # 오세아니아 축구 연맹
}

all_confederation = []

for confederation_key, kontinent_id in confederation_ids.items():
    for page in range(1, 21):
        url = f'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop/plus/ausrichtung/alle/spielerposition_id/alle/altersklasse/alle/jahrgang/0/land_id/0/kontinent_id/{kontinent_id}/yt0/Anzeigen/0//page/{page}?ajax=yw1'
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.content, 'html.parser')
        player_info = soup.find_all('tr', {'class': ['odd', 'even']})

        for info in player_info:
            player = info.find_all('td')
            all_confederation.append({
                'name': player[3].text.strip(),
                'position': player[4].text.strip(),
                'age': player[5].text.strip(),
                'nation': player[6].find('img').get('alt') if player[6].find('img') else '',
                'club': player[7].find('img').get('alt') if player[7].find('img') else '',
                'value': player[8].text.strip(),
                'confederation': confederation_key
                })

all_confederation_df = pd.DataFrame(all_confederation)
all_confederation_df = all_confederation_df.drop_duplicates()

all_confederation_df.to_csv('../data/transfermarkt_all_confederation.csv', index=False, encoding='utf-8-sig')
all_confederation_df

,name,position,age,nation,club,value,confederation
0,Kaoru Mitoma,Left Winger,28,Japan,Brighton & Hove Albion,€40.00m,UEFA
1,Min-jae Kim,Centre-Back,28,"Korea, South",Bayern Munich,€40.00m,UEFA
2,Abdukodir Khusanov,Centre-Back,21,Uzbekistan,Manchester City,€35.00m,UEFA
3,Takefusa Kubo,Right Winger,24,Japan,Real Sociedad,€30.00m,UEFA
4,Kang-in Lee,Right Winger,24,"Korea, South",Paris Saint-Germain,€25.00m,UEFA
...,...,...,...,...,...,...,...
2995,Illan Meslier,Goalkeeper,25,France,Leeds United,€16.00m,OFC
2996,Konstantin Tyukavin,Centre-Forward,23,Russia,Dynamo Moscow,€16.00m,OFC
2997,Finn Azaz,Attacking Midfield,24,Ireland,Middlesbrough FC,€16.00m,OFC
2998,Lloyd Kelly,Centre-Back,26,England,Juventus FC,€16.00m,OFC


4. 기타리그 별 선수가치

In [54]:
headers = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
}

league_codes = {
    'MLS': 'MLS1',  # 미국 리그
    'Championship': 'GB2', # 잉글랜드 2부리그
    'Eredivisie': 'NL1',  # 네덜란드 리그
    'Süper Lig': 'TR1',  # 튀르키예 리그
    'Scottish Premiership': 'SC1',  # 스코틀랜드 리그
    'Swiss SuperLeague': 'C1',  # 스위스 리그
    'Austria Bundesliga': 'A1',  # 오스트리아 리그
    'Danish Superligaen': 'DK1',  # 덴마크 리그
    'Saudi ProLeague': 'SA1',  # 사우디아라비아 리그
    'Liga Portugal': 'PO1',  # 포르투갈 리그
    'Russian PremierLiga': 'RU1',  # 러시아 리그
    'Brasileiro Série A': 'BRA1',  # 브라질 리그
    'Liga Profesional': 'ARGC',  # 아르헨티나 리그
    'Liga MX Apertura': 'MEXA',  # 멕시코 리그
    'Jupiler ProLeague': 'BE1',  # 벨기에 리그
    'SuperLeague1 Greece': 'GR1',  # 그리스 리그
    'A-League Men': 'AUS1',  # 호주 리그
    'J1 League': 'JAP1',  # 일본 리그
    'K-League1': 'RSK1',  # K리그
    'CSL': 'CSL'  # 중국 리그
}

other_leagues = []

for league_name, code in league_codes.items():
    for page in range(1, 5):
        url = f'https://www.transfermarkt.com/jumplist/marktwerte/wettbewerb/{code}/page/{page}?ajax=yw1'
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.content, 'html.parser')
        player_info = soup.find_all('tr', class_=['odd', 'even'])

        for info in player_info:
            try:
                inline_table = info.find('table', class_='inline-table')
                name = position = ''
                
                if inline_table:
                    trs = inline_table.find_all('tr')
                    name_tag = trs[0].find('a') if len(trs) > 0 else None
                    name = name_tag.text.strip() if name_tag else ''
                    position_td = trs[1].find('td') if len(trs) > 1 else None
                    position = position_td.text.strip() if position_td else ''

                tds = info.find_all('td', class_='zentriert')
                age = nation = club = ''
                
                for td in tds:
                    if td.text.strip().isdigit():
                        age = td.text.strip()
                    elif td.find('img') and 'flaggenrahmen' in td.find('img').get('class', []):
                        nation_img = td.find('img')
                        nation = nation_img['alt'] if nation_img and nation_img.has_attr('alt') else ''
                    elif td.find('img'):
                        club_img = td.find('img')
                        club = club_img['alt'] if club_img and club_img.has_attr('alt') else ''

                value_td = info.find('td', class_='rechts hauptlink')
                value_tag = value_td.find('a') if value_td else None
                value = value_tag.text.strip() if value_tag else value_td.text.strip() if value_td else ''

                other_leagues.append({
                    'name': name,
                    'position': position,
                    'age': age,
                    'nation': nation,
                    'club': club,
                    'value': value,
                    'league': league_name
                })

            except Exception as e:
                print(f"[Error] {e}")

        time.sleep(1)

other_leagues_df = pd.DataFrame(other_leagues)
other_leagues_df = other_leagues_df.drop_duplicates()
other_leagues_df = other_leagues_df[other_leagues_df['name'].str.strip() != '']

other_leagues_df.to_csv('../data/transfermarkt_other_leagues.csv', index=False, encoding='utf-8-sig')
other_leagues_df

,name,position,age,nation,club,value,league
0,Riqui Puig,Central Midfield,25,Spain,Los Angeles Galaxy,€20.00m,MLS
1,Lionel Messi,Right Winger,38,Argentina,Inter Miami CF,€18.00m,MLS
2,Emmanuel Latte Lath,Centre-Forward,26,Cote d'Ivoire,Atlanta United FC,€18.00m,MLS
3,Evander,Attacking Midfield,27,Brazil,FC Cincinnati,€14.00m,MLS
4,Kévin Denkey,Centre-Forward,24,Togo,FC Cincinnati,€14.00m,MLS
...,...,...,...,...,...,...,...
2275,Jin Cheng,Attacking Midfield,30,China,Zhejiang FC,€350k,CSL
2276,Qianglong Tao,Centre-Forward,23,China,Zhejiang FC,€350k,CSL
2277,Shiqin Wang,Left-Back,22,China,Zhejiang FC,€350k,CSL
2278,Rodrigo Henrique,Left Winger,32,Brazil,Meizhou Hakka,€350k,CSL


5. 최종 병합

-  all_positions_df + risingstars_df 병합과정

In [75]:
merged1_df = pd.concat([all_positions_df, risingstars_df], ignore_index=True)

# 두 데이터프레임에서 중복되는 행 찾기
duplicate_rows = pd.merge(all_positions_df, risingstars_df, on=['name', 'position', 'age', 'nation', 'club', 'value'])
print(f'all_positions 수: {len(all_positions_df)}, risingstars 수: {len(risingstars_df)}, 중복 데이터 수: {len(duplicate_rows)}')

merged1_df = merged1_df.drop_duplicates()
merged1_df.to_csv('../data/transfermarkt_merged1_df.csv', index=False, encoding='utf-8-sig')
merged1_df

all_positions 수: 2000, risingstars 수: 500, 중복 데이터 수: 293


,name,position,age,nation,club,value,confederation,league
0,Lamine Yamal,Right Winger,18,Spain,FC Barcelona,€200.00m,None,None
1,Erling Haaland,Centre-Forward,25,Norway,Manchester City,€180.00m,None,None
2,Kylian Mbappé,Centre-Forward,26,France,Real Madrid,€180.00m,None,None
3,Vinicius Junior,Left Winger,25,Brazil,Real Madrid,€170.00m,None,None
4,Bukayo Saka,Right Winger,23,England,Arsenal FC,€150.00m,None,None
...,...,...,...,...,...,...,...,...
2495,Maurice Krattenmacher,Attacking Midfield,19,Germany,Hertha BSC,€3.50m,None,None
2496,Max Dean,Centre-Forward,21,England,KAA Gent,€3.50m,None,None
2497,Charlie Hughes,Centre-Back,21,England,Hull City,€3.50m,None,None
2498,Cihan Çanak,Right Winger,20,Türkiye,Trabzonspor,€3.50m,None,None


- (all_positions_df + risingstars_df) + all_confederation_df 병합과정

In [ ]:
# 인덱스 초기화
all_confederation_df = all_confederation_df.reset_index(drop=True)
merged1_df = merged1_df.reset_index(drop=True)

# 병합 기준 key 컬럼
key_cols = ['name', 'position', 'age', 'nation', 'club', 'value']

# 컬럼 타입 맞추기
for col in key_cols:
    if col in all_confederation_df.columns and col in merged1_df.columns:
        if col == 'age':
            all_confederation_df[col] = pd.to_numeric(all_confederation_df[col], errors='coerce')
            merged1_df[col] = pd.to_numeric(merged1_df[col], errors='coerce')
        else:
            all_confederation_df[col] = all_confederation_df[col].astype(str)
            merged1_df[col] = merged1_df[col].astype(str)

# 컬럼 순서 정렬
columns_order = ['name', 'position', 'age', 'nation', 'club', 'value', 'confederation', 'league']
for col in columns_order:
    if col not in all_confederation_df.columns:
        all_confederation_df[col] = None
    if col not in merged1_df.columns:
        merged1_df[col] = None
all_confederation_df = all_confederation_df[columns_order]
merged1_df = merged1_df[columns_order]

# 병합
merged2_df = pd.concat([all_confederation_df, merged1_df], ignore_index=True)

# 중복 데이터 찾기
duplicate2_rows = pd.merge(all_confederation_df, merged1_df, on=key_cols)
print(f'all_confederation_df 수: {len(all_confederation_df)}, pos+rising 수: {len(merged1_df)}, 중복 데이터 수: {len(duplicate2_rows)}')

merged2_df = merged2_df.sort_values(by='confederation', ascending=False)
merged2_df = merged2_df.drop_duplicates(subset=key_cols)

merged2_df.to_csv('../data/transfermarkt_merged2_df.csv', index=False, encoding='utf-8-sig')
merged2_df

all_confederation_df 수: 2917, pos+rising 수: 2207, 중복 데이터 수: 1128


,name,position,age,nation,club,value,confederation,league
0,Kaoru Mitoma,Left Winger,28.0,Japan,Brighton & Hove Albion,€40.00m,UEFA,None
330,So Kawahara,Defensive Midfield,27.0,Japan,Kawasaki Frontale,€850k,UEFA,None
343,Ryuya Nishio,Centre-Back,24.0,Japan,Cerezo Osaka,€850k,UEFA,None
342,Motohiko Nakajima,Second Striker,26.0,Japan,Cerezo Osaka,€850k,UEFA,None
341,Hirokazu Ishihara,Right-Back,26.0,Japan,Urawa Red Diamonds,€850k,UEFA,None
...,...,...,...,...,...,...,...,...
5119,Maurice Krattenmacher,Attacking Midfield,19.0,Germany,Hertha BSC,€3.50m,NaN,NaN
5120,Max Dean,Centre-Forward,21.0,England,KAA Gent,€3.50m,NaN,NaN
5121,Charlie Hughes,Centre-Back,21.0,England,Hull City,€3.50m,NaN,NaN
5122,Cihan Çanak,Right Winger,20.0,Türkiye,Trabzonspor,€3.50m,NaN,NaN


- (all_positions_df + risingstars_df + all_confederation_df) + other_leagues_df 병합과정

In [ ]:
# 인덱스 초기화
other_leagues_df = other_leagues_df.reset_index(drop=True)
merged2_df = merged2_df.reset_index(drop=True)

# 병합 기준 key 컬럼
key_cols = ['name', 'position', 'age', 'nation', 'club', 'value']

# 컬럼 타입 맞추기
for col in key_cols:
    if col in merged2_df.columns and col in other_leagues_df.columns:
        if col == 'age':
            merged2_df[col] = pd.to_numeric(merged2_df[col], errors='coerce')
            other_leagues_df[col] = pd.to_numeric(other_leagues_df[col], errors='coerce')
        else:
            merged2_df[col] = merged2_df[col].astype(str)
            other_leagues_df[col] = other_leagues_df[col].astype(str)

# 중복 데이터 찾기
duplicate3_rows = pd.merge(merged2_df, other_leagues_df, on=key_cols)

# 결과 출력
print(f'other_leagues_df 수: {len(other_leagues_df)}, pos+rising+confs 수: {len(merged2_df)}, 중복 데이터 수: {len(duplicate3_rows)}')

all_players_value_df = pd.concat([merged2_df, other_leagues_df], ignore_index=True)
all_players_value_df = all_players_value_df.drop_duplicates(subset=key_cols)

all_players_value_df.to_csv('../data/transfermarkt_all_players_values_df.csv', index=False, encoding='utf-8-sig')
all_players_value_df

other_leagues_df 수: 2000, pos+rising+confs 수: 3996, 중복 데이터 수: 1027


,name,position,age,nation,club,value,confederation,league
0,Kaoru Mitoma,Left Winger,28.0,Japan,Brighton & Hove Albion,€40.00m,UEFA,None
1,So Kawahara,Defensive Midfield,27.0,Japan,Kawasaki Frontale,€850k,UEFA,None
2,Ryuya Nishio,Centre-Back,24.0,Japan,Cerezo Osaka,€850k,UEFA,None
3,Motohiko Nakajima,Second Striker,26.0,Japan,Cerezo Osaka,€850k,UEFA,None
4,Hirokazu Ishihara,Right-Back,26.0,Japan,Urawa Red Diamonds,€850k,UEFA,None
...,...,...,...,...,...,...,...,...
5991,Jin Cheng,Attacking Midfield,30.0,China,Zhejiang FC,€350k,None,CSL
5992,Qianglong Tao,Centre-Forward,23.0,China,Zhejiang FC,€350k,None,CSL
5993,Shiqin Wang,Left-Back,22.0,China,Zhejiang FC,€350k,None,CSL
5994,Rodrigo Henrique,Left Winger,32.0,Brazil,Meizhou Hakka,€350k,None,CSL
